In [1]:
import json
import pandas as pd
import secrets

In [2]:
def assign_rank(x):
    if pd.isna(x):
        return None
    return int(x // .1) + 1

def process_csv_ballots(csv_path, round_name):

    raw_voting_data = pd.read_csv(csv_path)

    dataframes = []
    for _, row in raw_voting_data.iterrows():
        if row['Status'] != 'SUBMITTED' and not row['Badgeholder']:
            continue
        if not isinstance(row['Payload'], str):
            continue

        payload = json.loads(row['Payload'])
        projects_payload = payload['project_allocations']
        
        addr = row['Address'].lower()
        processed_voting_data = []
        for votes in projects_payload:
            project_id = list(votes.keys())[0].lower()
            project_vote = list(votes.values())[0]
            processed_voting_data.append({
                'voter_address': addr,
                'project_id': project_id,
                'vote': project_vote
            })
        voter_dataframe = pd.DataFrame(processed_voting_data)
        dataframes.append(voter_dataframe)
        
    dataframe = pd.concat(dataframes, axis=0, ignore_index=True)
    #dataframe['project_score'] = pd.to_numeric(dataframe['vote']).rank(pct=True).apply(assign_rank)
    dataframe['round'] = round_name
    return dataframe

ballot_paths = [
    '../../../analysis/optimism/retrofunding5/data/_local/rf5_votes_final.csv',
    '../../../analysis/optimism/retrofunding6/data/_local/rf6_votes.csv'
]
voter_paths = [
    '../../../analysis/optimism/retrofunding5/data/_local/rf5_voters.csv',
    '../../../analysis/optimism/retrofunding6/data/_local/rf6_voters.csv',
]

In [3]:
df5 = pd.read_csv(voter_paths[0])
df5['voter_tags'] = df5.apply(lambda x: [x['Expertise group'], x['Voter Type']], axis=1)
df5['voter_address'] = df5['Wallet'].str.lower()
df5 = df5[['voter_address', 'voter_tags']]
df5 = process_csv_ballots(ballot_paths[0], 'RF5').merge(df5, on='voter_address')

voter_scramble = {v: f"0x{secrets.token_hex(20)}" for v in df5['voter_address'].unique()}
project_scramble = {p: f"0x{secrets.token_hex(32)}" for p in df5['project_id'].unique()}
df5['voter_address'] = df5['voter_address'].map(voter_scramble)
df5['project_id'] = df5['project_id'].map(project_scramble)

df5.tail(5)

,voter_address,project_id,vote,round,voter_tags
2921,0xe2d2f1b464a1085042f1b862cfbd636e453c1b35,0xa16daacf0ffbafdb2890e502d4a6e72012826cedccb9...,1.45,RF5,"[Expert, Guest Voter]"
2922,0xe2d2f1b464a1085042f1b862cfbd636e453c1b35,0xe869e7912c9685c36516c289849d90f031ea5022672a...,1.4,RF5,"[Expert, Guest Voter]"
2923,0xe2d2f1b464a1085042f1b862cfbd636e453c1b35,0x4002ae250d9dc86c32193d9ee3b879d9f9d08c23896c...,1.36,RF5,"[Expert, Guest Voter]"
2924,0xe2d2f1b464a1085042f1b862cfbd636e453c1b35,0xbd9d4fa4a54676ae08d7c71f92c5f0178f9b8385bd92...,1.31,RF5,"[Expert, Guest Voter]"
2925,0xe2d2f1b464a1085042f1b862cfbd636e453c1b35,0xb580e83b5ae96478d407c187e8b7c1decc178a57fe13...,1.27,RF5,"[Expert, Guest Voter]"


In [4]:
df6 = pd.read_csv(voter_paths[1])
df6['voter_tags'] = df6.apply(lambda x: [x['Voter Type']], axis=1)
df6['voter_address'] = df6['Wallet'].str.lower()
df6 = df6[['voter_address', 'voter_tags']]
df6 = process_csv_ballots(ballot_paths[1], 'RF6').merge(df6, on='voter_address')

voter_scramble = {v: f"0x{secrets.token_hex(20)}" for v in df6['voter_address'].unique()}
project_scramble = {p: f"0x{secrets.token_hex(32)}" for p in df6['project_id'].unique()}
df6['voter_address'] = df6['voter_address'].map(voter_scramble)
df6['project_id'] = df6['project_id'].map(project_scramble)

df6.tail(5)

,voter_address,project_id,vote,round,voter_tags
4165,0xf7b0f53c2d5759e96419f82e868cf693c64da2f7,0xe7c034dbeaf9f0fb0c62fa12c2a40d3c8ac8101a49e0...,0.58,RF6,[Citizen]
4166,0xf7b0f53c2d5759e96419f82e868cf693c64da2f7,0x40f987f74012feda6282df4e8f4e80d7f83cfb501fc3...,0.57,RF6,[Citizen]
4167,0xf7b0f53c2d5759e96419f82e868cf693c64da2f7,0x36232e0512e9dbdf7670f65998293e5a7a8857b0d928...,0.56,RF6,[Citizen]
4168,0xf7b0f53c2d5759e96419f82e868cf693c64da2f7,0x7f4193e558d77058dde2cd0e5f7d94c7df7da05ef054...,0.55,RF6,[Citizen]
4169,0xf7b0f53c2d5759e96419f82e868cf693c64da2f7,0xec56b0f38fc33f219ec3f468e6c320802f4aa5c65bbe...,None,RF6,[Citizen]


In [5]:
df = pd.concat([df5,df6], axis=0, ignore_index=True)
df.to_csv('rf_anon_votes.csv')